In [22]:
import pandas as pd
import numpy as np
import psycopg2
import boto3
import configparser
import seaborn as sb

## Configuración de la base de datos

In [15]:
config = configparser.ConfigParser()
config.read('proyecto.cfg')

['proyecto.cfg']

## Conexión a base de datos

In [16]:
import sql_queries

try:
    db_conn = psycopg2.connect(
        database = config.get('RDS', 'DB_NAME'),
        user = config.get('RDS', 'DB_USER'),
        password = config.get('RDS', 'DB_PASSWORD'),
        host = config.get('RDS', 'DB_HOST'),
        port = config.get('RDS', 'DB_PORT')
    )
    cursor = db_conn.cursor()
    cursor.execute(sql_queries.DDL_QUERY)
    db_conn.commit()
    print("Base de datos creada exitosamente.")
except Exception as ex:
    print("ERROR: Error al crear la base de datos.")
    print(ex)

Base de datos creada exitosamente.


## Carga de datos

In [17]:
def insertDataToSQL(data_dict, table_name):
     postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{config.get('RDS','DB_HOST')}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""    
     df_data = pd.DataFrame.from_records(data_dict)
     try:
          response = df_data.to_sql(table_name, postgres_driver, index=False, if_exists='append')
          print(f'Se han insertado {response} nuevos registros.' )
     except Exception as ex:
          print(ex)

In [18]:
import csv

sql_data = [*csv.DictReader(open('Video_Games_Sales_as_at_22_Dec_2016.csv'))]

insertDataToSQL(
    sql_data,
    'gamesales'
)

Se han insertado 719 nuevos registros.


## Análisis de los datos

In [21]:
sql_query = 'SELECT * FROM gamesales'
postgres_driver = f"""postgresql://{config.get('RDS', 'DB_USER')}:{config.get('RDS', 'DB_PASSWORD')}@{config.get('RDS','DB_HOST')}:{config.get('RDS', 'DB_PORT')}/{config.get('RDS', 'DB_NAME')}"""

#Carga de los datos a un dataframe
gamesales = pd.read_sql(sql_query, postgres_driver)